### URL 1개에 해당해서 파일 크롤링을 하는 방법

In [16]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [26]:
# 사이트 접속 차단을 받지 않기 위한 header 지정
header = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'
}

# 기업마당의 해시코드를 저장해놓은 dictionary
hashcode_dict = {'01':'금융',
                '02':'기술',
                '03':'인력',
                '07':'수출',
                '08':'내수',
                '09':'창업',
                '10':'경영',
                '12':'기타'}

In [28]:
def download_file(URL, header=header):
    '''지원사업 공고에 있는 파일을 다운로드하는 함수'''
    # bs4로 해당 URL의 html 구조 파싱
    response = requests.get(URL, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 다운로드 파일 링크를 담을 리스트 / 파일명을 담을 빈 문자열 선언
    down_link, down_title = [], ''

    # '본문출력파일'의 파일만 다운로드 하도록 수행하는 반복문 + 조건문
    for idx in range(len(soup.select('div.attached_file_list h3'))):
        if soup.select('div.attached_file_list h3')[idx].text == '본문출력파일':
            down_link = soup.select('div.attached_file_list div.right_btn')[idx].select('a')
            file_path = soup.select_one('div.category span').text

            # 다운 받을 공고문 파일의 이름을 공고문의 title로 바꿔서 정리
            down_title = soup.select_one('h2.title').text + '.' + soup.select('div.attached_file_list div.file_name')[idx].text.split('.')[-1]
        else:
            continue

    # 다운로드 파일링크는 내부 url이기 때문에 앞에 도메인 주소를 붙여줘야 함.
    base_url = 'https://www.bizinfo.go.kr'
    down_url = base_url + down_link[1].attrs['href']
    

    # 다운로드된 파일을 저장할 파일 경로
    save_path = f'C:/Users/every/Desktop/Project/team_project/SeSAC_Final/AI-for-Creating-Business-Plan-Drafts/{file_path}/{down_title}'

    # HTTP 요청을 보내 파일 다운로드
    response = requests.get(down_url, stream=True)

    # 응답이 성공했는지 확인
    if response.status_code == 200:
        with open(save_path, "wb") as file:
            for chunk in response.iter_content(1024):  # 1024바이트씩 저장
                file.write(chunk)
    else:
        print(f"파일 다운로드 실패. 상태 코드: {response.status_code}")


def fetch_links(hashcode, header=header):
    '''기업마당의 메인에 있는 공고 각각의 URL을 리스트로 반환'''

    # 기업마당의 공고 URL은 BASE_URL + td class='txt_l a'의 형식으로 되어 있음
    BASE_URL = 'https://www.bizinfo.go.kr/web/lay1/bbs/S1T122C128/AS/74/'
    idx = 1         # 페이지를 넘기기 위한 인덱스
    links = []      # 링크를 정리해둘 리스트 선언

    # 공고가 없는 페이지가 나올 때까지 반복문
    while True:
        URL = f'https://www.bizinfo.go.kr/web/lay1/bbs/S1T122C128/AS/74/list.do?hashCode={hashcode}&cpage={idx}'
        response = requests.get(URL, headers=header)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 공고가 없는 페이지가 나올 시 반복문 탈출
        if len(soup.select('div.sub_cont tr')) == 2:
            break
        
        # links에 URL 저장
        # 해당 페이지에 있는 전체 URL에 대하여 실행
        links += [BASE_URL + link.attrs['href'] for link in soup.select('td.txt_l a')]

        # 페이지 넘김
        idx += 1

    return links

In [29]:
for link in tqdm(fetch_links('01')):
    download_file(link)

100%|██████████| 146/146 [02:33<00:00,  1.05s/it]
